## 準備

1. KaggleのTitanic Datasetは必要なのでダウンロードしてGoogle Driveに上げる。 \
https://www.kaggle.com/c/titanic/data よりダウンロードが可能(Kaggleアカウントが必要) 

以下を`drive/MyDrive/Kaggle` 下に配置する。
- gender_submission.csv
- train.csv
- test.csv

2. ドライブをマウントする。 \
左のフォルダのアイコンからやる。

## 以下コード

### データの読み込み

In [ ]:
# 必要なものをインストール
!pip install matplotlib==3.3.3
!pip install category_encoders

In [ ]:
# データセットが入っているフォルダに移動
%cd /content/drive/MyDrive/Kaggle

/content/drive/MyDrive/Kaggle


In [ ]:
# 必要なライブラリをインストール
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import category_encoders as ce

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# 学習用データを読み込む
# 形状を確認
# 最初の5行を見る。
---write your code(3 rows)---

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# テストデータを読み込む
# 形状を確認
# 最後の5行を見る。
---write your code(3 rows)---

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
## 型を確認する。
---write your code---

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [ ]:
# 学習データの欠損値の数を確認する。
---write your code---
print('-'*40)
# テストデータの欠損値の数を確認する。
---write your code---

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------------------------------------
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


## 前処理

データがobject型のままではモデルが処理できないのでエンコーディングを行う。OneHot Encodingを用いる。 

また、ニューラルネットワークでは欠損値をそのまま扱うことができないので補填する。今回は、欠損値は全体の平均値で補填してみる。

なお、trainとtestでラベルの相違があったら困るので一度全データを結合してからエンコーディングする。

In [ ]:
## trainとtestを結合する。
---write your code---

In [ ]:
# 欠損値を補填する。
null_columns = ['Age', 'Cabin']
df_cat.fillna(df_cat.mean(), inplace=True)

In [ ]:
# object型のcolumnを定義する。(NameとTicketは種類が多すぎるので削除)
obj_columns = ['Sex', 'Cabin', 'Embarked']
# OneHotEncodingのインスタンスを作成する。(category_encodersではOneHotEncoderという名前である。)
oe = ---
# onehot encodingをdf_catに適用する。
df_cat_oe = ---
df_cat_oe.head()

,PassengerId,Survived,Pclass,Name,Sex_1,Sex_2,Age,SibSp,Parch,Ticket,Fare,Cabin_1,Cabin_2,Cabin_3,Cabin_4,Cabin_5,Cabin_6,Cabin_7,Cabin_8,Cabin_9,Cabin_10,Cabin_11,Cabin_12,Cabin_13,Cabin_14,Cabin_15,Cabin_16,Cabin_17,Cabin_18,Cabin_19,Cabin_20,Cabin_21,Cabin_22,Cabin_23,Cabin_24,Cabin_25,Cabin_26,Cabin_27,Cabin_28,Cabin_29,...,Cabin_152,Cabin_153,Cabin_154,Cabin_155,Cabin_156,Cabin_157,Cabin_158,Cabin_159,Cabin_160,Cabin_161,Cabin_162,Cabin_163,Cabin_164,Cabin_165,Cabin_166,Cabin_167,Cabin_168,Cabin_169,Cabin_170,Cabin_171,Cabin_172,Cabin_173,Cabin_174,Cabin_175,Cabin_176,Cabin_177,Cabin_178,Cabin_179,Cabin_180,Cabin_181,Cabin_182,Cabin_183,Cabin_184,Cabin_185,Cabin_186,Cabin_187,Embarked_1,Embarked_2,Embarked_3,Embarked_4
0,1,0.0,3,"Braund, Mr. Owen Harris",1,0,22.0,1,0,A/5 21171,7.2500,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,38.0,1,0,PC 17599,71.2833,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,1.0,3,"Heikkinen, Miss. Laina",0,1,26.0,0,0,STON/O2. 3101282,7.9250,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,35.0,1,0,113803,53.1000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,5,0.0,3,"Allen, Mr. William Henry",1,0,35.0,0,0,373450,8.0500,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


## モデル構築

今回は全結合ニューラルネットワーク(MLP)を用いる。
MLPはLightGBMと比較すると精度は劣るが、別の角度からの分析が可能になるため、アンサンブルをする際のモデルの一つとしては有用である。
MLPの実装はkerasを用いるのが楽である。 \
なお、MLPでは、値のスケーリングを行う。

また、本ノートブックの最後にPytorchでの実装方法も紹介する。

In [ ]:
## df_cat_oeをtrainとtestに分離する。
---write your code(2 rows)---

In [ ]:
# 学習に用いない特徴を定義する。
drop_cols = ['PassengerId', 'Survived', 'Name', 'Ticket']
# 学習データとテストデータを定義する(X, y, X_test)。
---write your code(3 rows)---

In [ ]:
from sklearn.model_selection import train_test_split
# 評価用データを作成する。一旦HoldOut法で行う。(random_stateは42にする。)
---write your code---

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
# モデルの構築をする。
def build_model(hid_dim=256,
                out_dim=1,
                hid_activation='relu',
                out_activation='sigmoid',
                optimizer='SGD',
                loss='binary_crossentropy',
                metrics=['accuracy'],
                drop_rate=0.1
                ):
  model = Sequential([
      Dense(hid_dim),
      Dropout(drop_rate),
      Activation(hid_activation),
      Dense(out_dim),
      Activation(out_activation),
  ])
  model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

  return model

# モデルを読み込む
model = build_model()

In [ ]:
---上のコードを写経してみよう---

In [ ]:
from sklearn.preprocessing import StandardScaler

# 入力データを標準化する。
sca = StandardScaler()
# X_trainにscalerをfitさせ、適用する。
X_train_sca = sca.fit_transform(X_train)
# X_valにscalerを適用する。
X_val_sca = sca.transform(---)
---write your code---  # X_testにscalerを適用する。

In [ ]:
# 学習
model.fit(X_train_sca, y_train, 
          batch_size=16,
          epochs=20,
          verbose=1,
          validation_data=(X_val_sca, y_val))

Epoch 1/20
45/45 [==============================] - 1s 7ms/step - loss: 0.7447 - accuracy: 0.6124 - val_loss: 0.6338 - val_accuracy: 0.7430
Epoch 2/20
45/45 [==============================] - 0s 2ms/step - loss: 0.6327 - accuracy: 0.7570 - val_loss: 0.5772 - val_accuracy: 0.7877
Epoch 3/20
45/45 [==============================] - 0s 3ms/step - loss: 0.5557 - accuracy: 0.7893 - val_loss: 0.5413 - val_accuracy: 0.7933
Epoch 4/20
45/45 [==============================] - 0s 2ms/step - loss: 0.5080 - accuracy: 0.8160 - val_loss: 0.5159 - val_accuracy: 0.7821
Epoch 5/20
45/45 [==============================] - 0s 2ms/step - loss: 0.4758 - accuracy: 0.8076 - val_loss: 0.4986 - val_accuracy: 0.7933
Epoch 6/20
45/45 [==============================] - 0s 2ms/step - loss: 0.4543 - accuracy: 0.8301 - val_loss: 0.4866 - val_accuracy: 0.7877
Epoch 7/20
45/45 [==============================] - 0s 2ms/step - loss: 0.4292 - accuracy: 0.8329 - val_loss: 0.4778 - val_accuracy: 0.7877
Epoch 8/20
45/45 [==

### valデータの評価

In [ ]:
from sklearn.metrics import accuracy_score
# valデータを予測する。(スケーリングしたデータを使おう。)
val_probs = model.predict(---)
# 形状を(179)にする
val_probs = val_probs.squeeze()
print(val_probs)

[0.20895812 0.1490016  0.12488136 0.8049522  0.76732564 0.89083207
 0.703824   0.12006292 0.72607356 0.8492006  0.982353   0.07817498
 0.40458363 0.17997643 0.14157316 0.88455015 0.3332038  0.70387745
 0.16118255 0.34898064 0.11502355 0.24285975 0.5804823  0.12557283
 0.10477427 0.11971968 0.24777901 0.15054154 0.14041302 0.601123
 0.12774211 0.617944   0.2756224  0.6030872  0.13053653 0.19772586
 0.36540854 0.703824   0.8953921  0.10191518 0.19635981 0.0955058
 0.10197824 0.13441548 0.66564405 0.15413523 0.12787229 0.11789876
 0.11458966 0.30604815 0.7760564  0.8480347  0.07468021 0.9559212
 0.08521986 0.8710158  0.14699572 0.40189213 0.7879841  0.7206812
 0.12207681 0.7965394  0.766564   0.25356096 0.13441548 0.78196156
 0.24315226 0.10402179 0.2894792  0.98286724 0.75051194 0.9986757
 0.4318151  0.88606524 0.11719325 0.08738551 0.7027503  0.90659237
 0.78391516 0.52321887 0.0858672  0.96238244 0.8963661  0.13440982
 0.31388158 0.20978263 0.98771584 0.89817256 0.21714374 0.1016514
 0

モデルの出力は(targetの数, 1)で出力される。
これを、以下のルールで変更する。 \
1. 出力の形状を(targetの数)に直す(np.squeeze()を用いる。)
2. thershold=0.5
3. 値がthresholdを下回ったら0
4. それ以上の場合は1 \

In [ ]:
thershold = 0.5
# 予測を二値化する。
val_preds = ---

Accuracyスコアで精度を計算する。

In [ ]:
from sklearn.metrics import accuracy_score
# スコアを出力
---write your code---

0.7821229050279329


### テストデータの予測とKaggleへの提出ファイルの作成。

In [ ]:
# valデータと同じように予測
---write your code---
# test_probsの形状を直す。
---write your code---
# valデータと同じように予測を二値にする。
thershold = 0.5
---write your code---

In [ ]:
# submissionのサンプルファイルを読み込む
---write your code---
# サンプルのtargetカラムをtest_predsに置き換える。
---write your code---

In [ ]:
# 提出 (パスは submit/my_submit3.csv とする。)
---write your code---

### Cross Validationを作ろう

今回は、穴埋め形式ではなく1から実装してもらうことにする。 \
LightGBMのCross Validationのコードをコピペして、ニューラルネットワーク専用コードに編集すれば効率良く実装できる。(もちろん一から実装した方が勉強にはなる。)

In [ ]:
from sklearn.model_selection import StratifiedKFold

---write your code(lots of rows)---

epoch1
0.8044692737430168
----------------------------------------
epoch2
0.8202247191011236
----------------------------------------
epoch3
0.8089887640449438
----------------------------------------
epoch4
0.8539325842696629
----------------------------------------
epoch5
0.8370786516853933
----------------------------------------
oof score :  0.8249158249158249


In [ ]:
# 提出 (パスは submit/my_submit4.csv とする)
---write your code(3 rows)---

## Pytorchで実装しよう

Pytorchは学術研究で広く用いられるフレームワークである。 \
実装を1からカスタマイズする必要があるので簡単な全結合ネットワークだとkerasの方が何かと便利だが、高度なディープラーニングなどで真価を発揮するため、実装してみる。

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# Pytorchモデルを定義する。
class MLP(nn.Module):

  def __init__(self, input_dim, hid_dim=256, out_dim=1, drop_rate=0.1):
    # nn.Moduleクラスを継承する。
    super(MLP, self).__init__()
    # モデルのパーツを定義する。
    self.layer = nn.Sequential(
      nn.Linear(input_dim, hid_dim),
      nn.Dropout(drop_rate),
      nn.ReLU(),
      nn.Linear(hid_dim, out_dim),
      nn.Sigmoid()
    )

  def forward(self, x):
    # モデルの順伝搬を行う。
    return self.layer(x).squeeze()


In [ ]:
---上のコードを写経しよう---

In [ ]:
# モデルの挙動を確認
sample_x = torch.randn([16, 198])
print(sample_x.size(1))
sample_model = MLP(sample_x.size(1))
sample_out = sample_model(sample_x)
print(sample_out.shape)

198
torch.Size([16])


In [ ]:
# pytorchのデータセットを作る。
class MLPDataset(Dataset):

  def __init__(self, X, y=None, test=False):
    super().__init__()
    self.X = X
    self.test = test
    # testデータの時はyは存在しないので定義しない。
    if not test:
      self.y = y

  def __len__(self): 
    return len(self.X)

  def __getitem__(self, idx):
    # X, yをtorch.Tensorに変換
    X_torch = torch.Tensor(self.X.values)
    if not self.test:
      y_torch = torch.Tensor(self.y)
      return [X_torch[idx], y_torch[idx]]
    else:
      return [X_torch[idx]]


In [ ]:
---上のコードを写経しよう---

In [ ]:
# Datasetの動作確認
sample_dataset = MLPDataset(X, y)
sample_dataset[0]

[tensor([ 3.0000,  1.0000,  0.0000, 22.0000,  1.0000,  0.0000,  7.2500,  1.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

In [ ]:
# dataloaderを定義
sample_dataloader = DataLoader(sample_dataset, batch_size=16, shuffle=True)
for sd in sample_dataloader:
  print(sd[0].shape)
  print(sd[1].shape)
  break

torch.Size([16, 198])
torch.Size([16])


## Pytorchで学習

In [ ]:
def torch_train(model, train_dataset, valid_dataset, criterion, optimizer,
                epochs=20, batch_size=16, verbose=0, threshold=0.5):
  # pytorchで学習を行う。
  for epoch in range(epochs):
    # 学習モードにする。
    model.train()
    # train_dataloaderを定義する。
    ---write your code---
    # train_lossを記録する。
    train_loss = 0.0
    for td in train_dataloader:
      # optimizerの勾配をリセット
      optimizer.zero_grad()
      # inputとtargetを定義
      input = td[0]
      target = td[1]
      # 順伝搬
      ---write your code---
      # lossを求める。
      loss = criterion(---, ---)
      # train_lossを加算する。
      train_loss += loss.item()
      # 逆伝搬
      loss.backward()
      # optimizerを更新
      optimizer.step()

    # 評価モードにする。
    model.eval()
    # valid_dataloderを定義する
    ---write your code---
    # outputとtargetを格納するリスト(accuracyの計算で使う。)
    targets = []
    outputs = []
    # lossを記録する。
    valid_loss = 0.0
    # 勾配計算をしないようにする。
    with torch.no_grad():
      for vd in ---:
        input = ---
        target = ---
        # 順伝搬
        ---write your code---
        # lossを求める。
        ---write your code---
        # target, outputを記録する。
        targets.extend(target.numpy())
        outputs.extend(output.numpy())
        # valid_lossを加算する。
        ---write your code---
    
    # outputsを特定の閾値を利用して二値化する。(outputsはlist型なのでnumpy配列に変換しよう。)
    ---write your code---
    # valid_accuracyを求める。
    ---write your code---

    # 特定エポックで出力する。
    if verbose != 0 and (epoch) % verbose == 0:
      print(f'epoch{epoch+1}')
      print("train loss : {}        valid_loss : {}       valid_accuracy : {}".format(train_loss, valid_loss, valid_accuracy))

  return model

def torch_predict(model, test_dataset, batch_size=16, threshold=0.5):
  # pytorchで学習したモデルを利用して予測を行う。
  # 評価モードにする。
  ---write your code---
  # test_dataloaderを定義する。
  ---write your code---
  test_probs = []  # 予測値を格納するリスト
  # 勾配計算をしないようにする。
  ---:
    for ted in ---:
      input = ---
      output = ---
      # test_probsを追加する。
      test_probs.extend(output.numpy())
  # test_probsを特定の閾値で二値化
  ---write your code---
  
  return test_preds

In [ ]:
from torch.nn import BCELoss
from torch.optim import SGD, Adam
# ハイパーパラメータを設定
EPOCHS = 20
HID_DIM = 256
DROP_RATE = 0.1
BATCH_SIZE=16
feature_dim = X.shape[1]

# 評価用データを作成する。一旦HoldOut法で行う。
---write your code---

# datasetを定義する。(train_dataset, valid_dataset, test_dataset)
---write your code(3 rows)---

# モデルを定義する。
---write your code---
# 損失関数と最適化関数を定義する。
---write your code(2 rows)---

# modelを学習する。
model = torch_train(model, ---, ---, ---, ---,
                    epochs=---, batch_size=---, verbose=1)
# 予測する。
test_preds = torch_predict(---, ---, batch_size=---)
print(test_preds)

epoch1
train loss : 38.818934708833694        valid_loss : 6.591416746377945       valid_accuracy : 0.7094972067039106
epoch2
train loss : 25.815645277500153        valid_loss : 6.410161793231964       valid_accuracy : 0.7597765363128491
epoch3
train loss : 23.17527887225151        valid_loss : 5.387544512748718       valid_accuracy : 0.7821229050279329
epoch4
train loss : 21.392673566937447        valid_loss : 5.36792066693306       valid_accuracy : 0.7877094972067039
epoch5
train loss : 21.523722514510155        valid_loss : 5.564299002289772       valid_accuracy : 0.8100558659217877
epoch6
train loss : 21.53463687002659        valid_loss : 6.198324680328369       valid_accuracy : 0.7430167597765364
epoch7
train loss : 21.70707356929779        valid_loss : 5.7540357410907745       valid_accuracy : 0.7988826815642458
epoch8
train loss : 21.872661232948303        valid_loss : 5.548520117998123       valid_accuracy : 0.7653631284916201
epoch9
train loss : 20.02983033657074        valid_

In [ ]:
# 提出 (パスは submit/my_submit5.csv とする。)
---write your code(3 rows)---

## Cross Validationを実装する。

pytorchの学習と予測は独自で、関数化してあるため実装は、そこまで難しくないはずである。

In [ ]:
from sklearn.model_selection import StratifiedKFold

---write your code(lots of rows)---

epoch1
0.8100558659217877
----------------------------------------
epoch2
0.797752808988764
----------------------------------------
epoch3
0.7865168539325843
----------------------------------------
epoch4
0.8146067415730337
----------------------------------------
epoch5
0.8370786516853933
----------------------------------------
oof score :  0.8092031425364759


In [ ]:
# 提出 (パスは submit/my_submit6.csv とする。)
---write your code(3 rows)---

## 次にやること

- 次回のTabnetのチュートリアルに取り組もう
- PytorchのコードをGPUに対応させよう
- 特徴量エンジニアリングをしよう。
- 前回作成したLightGBMモデルとアンサンブルしてみよう。

## 参考サイト
- Kerasの公式ドキュメント : https://keras.io/ja/
- Pytorchの公式ドキュメント : https://pytorch.org/docs/stable/index.html
- Pytorchでテーブルデータを処理する方法 : https://towardsdatascience.com/deep-learning-using-pytorch-for-tabular-data-c68017d8b480